# False positives

author: steeve.laquitaine@epfl.ch  
last modified: 13-02-2024

**Method**:

* **delta_time (Δ𝑡)** = 1.3 ms: the time windows before and after the spike timestamp of a ground truth. When a the timestamp of a sorted unit falls within this time window, they coincide and the sorted timestamp is a hit.
* **chance level score**: see paper
* **dark (missed) units**: ground truth units with sorting accuracy below the chance agreement score (their best match with a sorted unit produce an agreement score below chance).
* **false positive units**: units which timestamps never hit the timestamps of the ground truth units wihin 50 microns of the probe: they never fall withing the delta_time window.

**New!**
* delta_time: 1.3 ms instead of 0.4 ms
* chance score: theoretically derived instead of 0.1
* false positive: units with all scores below chance 

**TODO**:
* do the same for the dense spontaneous recording: ensure that for each layer, we fully cover each layer only once, same as for neuropixels to enable a fair comparison.
Because of the multiple probe insertions we might cover a layer twice, which would oversample it and change the overall ratio of sorted unit qualities.



### Setup

Create or activate env `spikeinterf...`

In [1]:
%load_ext autoreload
%autoreload 2
import os 
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import spikeinterface as si

# set project path
proj_path = "/gpfs/bbp.cscs.ch/project/proj85/home/laquitai/preprint_2023/"
os.chdir(proj_path)

from src.nodes.utils import get_config
from src.nodes.analysis.failures import accuracy as acc
from src.nodes.metrics import quality
from src.nodes.metrics.quality import get_scores_for_dense_probe as gscdp
from src.nodes.metrics.quality import get_chance_for_dense_probe as gchdp
from src.nodes.metrics.quality import combine_quality_across_dense_probe as cqadb

# PARAMETERS
DUR = 600 # 10 minutes recording
DT = 1.3 # ms (optimized)
THR_GOOD = 0.8

# DATASETS

# NPX PROBE
# Synthetic (10m)
cfg_nb, _ = get_config("buccino_2020", "2020").values()
GT_nb_10m = cfg_nb["sorting"]["simulation"]["ground_truth"]["10m"]["output"]
KS4_nb_10m = cfg_nb["sorting"]["sorters"]["kilosort4"]["10m"]["output"]
KS3_nb_10m = cfg_nb["sorting"]["sorters"]["kilosort3"]["10m"]["output"]
KS2_5_nb_10m = cfg_nb["sorting"]["sorters"]["kilosort2_5"]["10m"]["output"]
KS2_nb_10m = cfg_nb["sorting"]["sorters"]["kilosort2"]["10m"]["output"]
KS_nb_10m = cfg_nb["sorting"]["sorters"]["kilosort"]["10m"]["output"]
HS_nb_10m = cfg_nb["sorting"]["sorters"]["herdingspikes"]["10m"]["output"]
REC_nb = cfg_nb["probe_wiring"]["full"]["output"]

# biophy spont (10m)
cfg_ns, _ = get_config("silico_neuropixels", "concatenated").values()
KS4_ns_10m = cfg_ns["sorting"]["sorters"]["kilosort4"]["10m"]["output"]
KS3_ns_10m = cfg_ns["sorting"]["sorters"]["kilosort3"]["10m"]["output"]
KS2_5_ns_10m = cfg_ns["sorting"]["sorters"]["kilosort2_5"]["10m"]["output"]
KS_ns_10m = cfg_ns["sorting"]["sorters"]["kilosort"]["10m"]["output"]
HS_ns_10m = cfg_ns["sorting"]["sorters"]["herdingspikes"]["10m"]["output"]
GT_ns_10m = cfg_ns["sorting"]["simulation"]["ground_truth"]["10m"]["output"]
REC_ns = cfg_ns["probe_wiring"]["full"]["output"]

# biophy evoked
cfg_ne, _ = get_config("silico_neuropixels", "stimulus").values()
KS4_ne_10m = cfg_ne["sorting"]["sorters"]["kilosort3"]["10m"]["output"]
KS3_ne_10m = cfg_ne["sorting"]["sorters"]["kilosort3"]["10m"]["output"]
KS2_ne_10m = cfg_ne["sorting"]["sorters"]["kilosort2"]["10m"]["output"]
KS_ne_10m = cfg_ne["sorting"]["sorters"]["kilosort"]["10m"]["output"]
HS_ne_10m = cfg_ne["sorting"]["sorters"]["herdingspikes"]["10m"]["output"]
GT_ne_10m = cfg_ne["sorting"]["simulation"]["ground_truth"]["10m"]["output"]
REC_ne = cfg_ne["probe_wiring"]["full"]["output"]

# DENSE PROBE 
# depth 1
cfg_ds1, _ = get_config("silico_horvath", "concatenated/probe_1").values()
K4_d1 = cfg_ds1["sorting"]["sorters"]["kilosort4"]["10m"]["output"]
K3_d1 = cfg_ds1["sorting"]["sorters"]["kilosort3"]["10m"]["output"]
K25_d1 = cfg_ds1["sorting"]["sorters"]["kilosort2_5"]["10m"]["output"]
K2_d1 = cfg_ds1["sorting"]["sorters"]["kilosort2"]["10m"]["output"]
K_d1 = cfg_ds1["sorting"]["sorters"]["kilosort"]["10m"]["output"]
H_d1 = cfg_ds1["sorting"]["sorters"]["herdingspikes"]["10m"]["output"]
R_d1 = cfg_ds1["probe_wiring"]["full"]["output"]
T_d1 = cfg_ds1["sorting"]["simulation"]["ground_truth"]["10m"]["output"]

# depth 2
cfg_ds2, _ = get_config("silico_horvath", "concatenated/probe_2").values()
K4_d2 = cfg_ds2["sorting"]["sorters"]["kilosort4"]["10m"]["output"]
K3_d2 = cfg_ds2["sorting"]["sorters"]["kilosort3"]["10m"]["output"]
K25_d2 = cfg_ds2["sorting"]["sorters"]["kilosort2_5"]["10m"]["output"]
K2_d2 = cfg_ds2["sorting"]["sorters"]["kilosort2"]["10m"]["output"]
K_d2 = cfg_ds2["sorting"]["sorters"]["kilosort"]["10m"]["output"]
H_d2 = cfg_ds2["sorting"]["sorters"]["herdingspikes"]["10m"]["output"]
R_d2 = cfg_ds2["probe_wiring"]["full"]["output"]
T_d2 = cfg_ds2["sorting"]["simulation"]["ground_truth"]["10m"]["output"]

# depth 3
cfg_ds3, _ = get_config("silico_horvath", "concatenated/probe_3").values()
K4_d3 = cfg_ds3["sorting"]["sorters"]["kilosort4"]["10m"]["output"]
K3_d3 = cfg_ds3["sorting"]["sorters"]["kilosort3"]["10m"]["output"]
K25_d3 = cfg_ds3["sorting"]["sorters"]["kilosort2_5"]["10m"]["output"]
K2_d3 = cfg_ds3["sorting"]["sorters"]["kilosort2"]["10m"]["output"]
K_d3 = cfg_ds3["sorting"]["sorters"]["kilosort"]["10m"]["output"]
H_d3 = cfg_ds3["sorting"]["sorters"]["herdingspikes"]["10m"]["output"]
R_d3 = cfg_ds3["probe_wiring"]["full"]["output"]
T_d3 = cfg_ds3["sorting"]["simulation"]["ground_truth"]["10m"]["output"]


# pre-computed qualities
quality_path = "/gpfs/bbp.cscs.ch/project/proj85/scratch/laquitai/4_preprint_2023/analysis/sorting_quality/sorting_quality.csv"

# FIGURE SETTINGS
# ticks
N_MJ_TCKS = 5
N_MN_TCKS = 11

# colors
# quality colors
cl = {"good": [0.7, 0.1, 0.1], # strong red
      "oversplitter": [0.6, 0.9, 0.6], # blue
      "overmerger": [0, 0.7, 1], # green
      "mixed: good + overmerger": np.array([[0.7, 0.1, 0.1], [0, 0.7, 1]]).mean(axis=0),
      "mixed: good + oversplitter": np.array([[0.7, 0.1, 0.1], [0.6, 0.9, 0.6]]).mean(axis=0),
      "mixed: overmerger + oversplitter": np.array([[0.6, 0.9, 0.6], [0, 0.7, 1]]).mean(axis=0),
      "mixed: good + overmerger + oversplitter": np.array([[0.7, 0.1, 0.1], [0, 0.7, 1],[0.6, 0.9, 0.6]]).mean(axis=0),
      "false positive": [0, 0, 0] # black
}

# axes
plt.rcParams["font.family"] = "Arial"
plt.rcParams["font.size"] = 6  # 5-7 with Nature neuroscience as reference
plt.rcParams["lines.linewidth"] = 0.5 # typically between 0.5 and 1
plt.rcParams["axes.linewidth"] = 0.5 #1
plt.rcParams["axes.spines.top"] = False
plt.rcParams["xtick.major.width"] = 0.5 #0.8 #* 1.3
plt.rcParams["xtick.minor.width"] = 0.5 #0.8 #* 1.3
plt.rcParams["ytick.major.width"] = 0.5 #0.8 #* 1.3
plt.rcParams["ytick.minor.width"] = 0.5 #0.8 #* 1.3
plt.rcParams["xtick.major.size"] = 3.5 * 1.1
plt.rcParams["xtick.minor.size"] = 2 * 1.1
plt.rcParams["ytick.major.size"] = 3.5 * 1.1
plt.rcParams["ytick.minor.size"] = 2 * 1.1
# legend
legend_cfg = {"frameon": False, "handletextpad": 0.5}
tight_layout_cfg = {"pad": 0.001}
LG_FRAMEON = False              # no legend frame

### (30s)Scores

* sorted single-units only
* 10 minutes recordings
* for the dense probe, sorted units filtered such that there is no overepresentation of a layer: from L1, L2/3 from depth 1, L4,5 (depth 2), L6 (depth 3)


In [10]:
# # KS4
# # npx-synthetic
# scores_nb_ks4, Sorting_nb_ks4, SortingTrue_nb = quality.get_score_single_unit(
#     KS4_nb_10m, GT_nb_10m, DT
# )
## npx-biophy.spont
scores_ns_ks4, Sorting_ns_ks4, SortingTrue_ns = quality.get_score_single_unit(
    KS4_ns_10m, GT_ns_10m, DT
)
## npx-biophy.evoked
# scores_ne_ks4, Sorting_ne_ks4, SortingTrue_ne = quality.get_score_single_unit(
#     KS4_ne_10m, GT_ne_10m, DT
# )
## dense-biophy.spont
(k4_sc, k4_so, k4_t) = gscdp(K4_d1, K4_d2, K4_d3, T_d1, T_d2, T_d3, DT)

# # KS3
# scores_nb_ks3, Sorting_nb_ks3, SortingTrue_nb = quality.get_score_single_unit(
#     KS3_nb_10m, GT_nb_10m, DT
# )
scores_ns_ks3, Sorting_ns_ks3, SortingTrue_ns = quality.get_score_single_unit(
    KS3_ns_10m, GT_ns_10m, DT
)
# scores_ne_ks3, Sorting_ne_ks3, SortingTrue_ne = quality.get_score_single_unit(
#     KS3_ne_10m, GT_ne_10m, DT
# )
(k3_sc, k3_so, k3_t) = gscdp(K3_d1, K3_d2, K3_d3, T_d1, T_d2, T_d3, DT)

# # KS2.5
# scores_nb_ks2_5, Sorting_nb_ks2_5, SortingTrue_nb = quality.get_score_single_unit(
#     KS3_nb_10m, GT_nb_10m, DT
# )
scores_ns_ks2_5, Sorting_ns_ks2_5, SortingTrue_ns = quality.get_score_single_unit(
    KS3_ns_10m, GT_ns_10m, DT
)
# scores_ne_ks2_5, Sorting_ne_ks2_5, SortingTrue_ne = quality.get_score_single_unit(
#     KS3_ne_10m, GT_ne_10m, DT
# )
(k25_sc, k25_so, k25_t) = gscdp(K25_d1, K25_d2, K25_d3, T_d1, T_d2, T_d3, DT)

# # KS2
# scores_nb_ks2, Sorting_nb_ks2, SortingTrue_nb = quality.get_score_single_unit(
#     KS3_nb_10m, GT_nb_10m, DT
# )
scores_ns_ks2, Sorting_ns_ks2, SortingTrue_ns = quality.get_score_single_unit(
    KS3_ns_10m, GT_ns_10m, DT
)
# scores_ne_ks2, Sorting_ne_ks2, SortingTrue_ne = quality.get_score_single_unit(
#     KS3_ne_10m, GT_ne_10m, DT
# )
(k2_sc, k2_so, k2_t) = gscdp(K2_d1, K2_d2, K2_d3, T_d1, T_d2, T_d3, DT)

# # KS
# scores_nb_ks, Sorting_nb_ks, SortingTrue_nb = quality.get_score_single_unit(
#     KS3_nb_10m, GT_nb_10m, DT
# )
scores_ns_ks, Sorting_ns_ks, SortingTrue_ns = quality.get_score_single_unit(
    KS3_ns_10m, GT_ns_10m, DT
)
# scores_ne_ks, Sorting_ne_ks, SortingTrue_ne = quality.get_score_single_unit(
#     KS3_ne_10m, GT_ne_10m, DT
# )
(K_sc, K_so, K_t) = gscdp(K_d1, K_d2, K_d3, T_d1, T_d2, T_d3, DT)

# # HS
# scores_nb_hs, Sorting_nb_hs, SortingTrue_nb = quality.get_score_single_unit(
#     KS3_nb_10m, GT_nb_10m, DT
# )
scores_ns_hs, Sorting_ns_hs, SortingTrue_ns = quality.get_score_single_unit(
    KS3_ns_10m, GT_ns_10m, DT
)
# scores_ne_hs, Sorting_ne_hs, SortingTrue_ne = quality.get_score_single_unit(
#     KS3_ne_10m, GT_ne_10m, DT
# )
(H_sc, H_so, H_t) = gscdp(H_d1, H_d2, H_d3, T_d1, T_d2, T_d3, DT)

### (9m)Chance scores

* sorted single-units only
* chance scores are theoretically derived

In [11]:
# pre-compute chance scores (parallelized)
# # KS4
# chance_nb_ks4 = quality.precompute_chance_score(
#     REC_nb, scores_nb_ks4, Sorting_nb_ks4, SortingTrue_nb, DUR, DT
# )
chance_ns_ks4 = quality.precompute_chance_score(
    REC_ns, scores_ns_ks4, Sorting_ns_ks4, SortingTrue_ns, DUR, DT
)
# chance_ne_ks4 = quality.precompute_chance_score(
#     REC_ne, scores_ne_ks4, Sorting_ne_ks4, SortingTrue_ne, DUR, DT
# )
## dense-biophy.spont
k4_ch = gchdp(DUR, DT, R_d1, R_d2, R_d3, **k4_sc, **k4_so, **k4_t)

# # KS3
# chance_nb_ks3 = quality.precompute_chance_score(
#     REC_nb, scores_nb_ks3, Sorting_nb_ks3, SortingTrue_nb, DUR, DT
# )
chance_ns_ks3 = quality.precompute_chance_score(
    REC_ns, scores_ns_ks3, Sorting_ns_ks3, SortingTrue_ns, DUR, DT
)
# chance_ne_ks3 = quality.precompute_chance_score(
#     REC_ne, scores_ne_ks3, Sorting_ne_ks3, SortingTrue_ne, DUR, DT
# )
k3_ch = gchdp(DUR, DT, R_d1, R_d2, R_d3, **k3_sc, **k3_so, **k3_t)

# # KS2_5
# chance_nb_ks2_5 = quality.precompute_chance_score(
#     REC_nb, scores_nb_ks2_5, Sorting_nb_ks2_5, SortingTrue_nb, DUR, DT
# )
chance_ns_ks2_5 = quality.precompute_chance_score(
    REC_ns, scores_ns_ks2_5, Sorting_ns_ks2_5, SortingTrue_ns, DUR, DT
)
# chance_ne_ks2_5 = quality.precompute_chance_score(
#     REC_ne, scores_ne_ks2_5, Sorting_ne_ks2_5, SortingTrue_ne, DUR, DT
# )
k25_ch = gchdp(DUR, DT, R_d1, R_d2, R_d3, **k25_sc, **k25_so, **k25_t)

# # KS2
# chance_nb_ks2 = quality.precompute_chance_score(
#     REC_nb, scores_nb_ks2, Sorting_nb_ks2, SortingTrue_nb, DUR, DT
# )
chance_ns_ks2 = quality.precompute_chance_score(
    REC_ns, scores_ns_ks2, Sorting_ns_ks2, SortingTrue_ns, DUR, DT
)
# chance_ne_ks2 = quality.precompute_chance_score(
#     REC_ne, scores_ne_ks2, Sorting_ne_ks2, SortingTrue_ne, DUR, DT
# )
k2_ch = gchdp(DUR, DT, R_d1, R_d2, R_d3, **k2_sc, **k2_so, **k2_t)

# # KS
# chance_nb_ks = quality.precompute_chance_score(
#     REC_nb, scores_nb_ks, Sorting_nb_ks, SortingTrue_nb, DUR, DT
# )
chance_ns_ks = quality.precompute_chance_score(
    REC_ns, scores_ns_ks, Sorting_ns_ks, SortingTrue_ns, DUR, DT
)
# chance_ne_ks = quality.precompute_chance_score(
#     REC_ne, scores_ne_ks, Sorting_ne_ks, SortingTrue_ne, DUR, DT
# )
K_ch = gchdp(DUR, DT, R_d1, R_d2, R_d3, **K_sc, **K_so, **K_t)

# # HS
# chance_nb_hs = quality.precompute_chance_score(
#     REC_nb, scores_nb_hs, Sorting_nb_hs, SortingTrue_nb, DUR, DT
# )
chance_ns_hs = quality.precompute_chance_score(
    REC_ns, scores_ns_hs, Sorting_ns_hs, SortingTrue_ns, DUR, DT
)
# chance_ne_hs = quality.precompute_chance_score(
#     REC_ne, scores_ne_hs, Sorting_ne_hs, SortingTrue_ne, DUR, DT
# )
H_ch = gchdp(DUR, DT, R_d1, R_d2, R_d3, **H_sc, **H_so, **H_t)

### (36m)Sorted unit quality

In [13]:
# (6m) qualify all sorted single-units
# (18m) KS4
df_ns_ks4 = quality.qualify_sorted_units(scores_ns_ks4, chance_ns_ks4, THR_GOOD)
# df_nb_ks4 = quality.qualify_sorted_units(scores_nb_ks4, chance_nb_ks4, THR_GOOD)
# df_ne_ks4 = quality.qualify_sorted_units(scores_ne_ks4, chance_ne_ks4, THR_GOOD)
df_ds_ks4 = cqadb(THR_GOOD, K4_d1, K4_d2, K4_d3, **k4_sc, **k4_ch)
# # KS3
df_ns_ks3 = quality.qualify_sorted_units(scores_ns_ks3, chance_ns_ks3, THR_GOOD)
# df_nb_ks3 = quality.qualify_sorted_units(scores_nb_ks3, chance_nb_ks3, THR_GOOD)
# df_ne_ks3 = quality.qualify_sorted_units(scores_ne_ks3, chance_ne_ks3, THR_GOOD)
df_ds_ks3 = cqadb(THR_GOOD, K3_d1, K3_d2, K3_d3, **k3_sc, **k3_ch)
# # KS2.5
df_ns_ks2_5 = quality.qualify_sorted_units(scores_ns_ks2_5, chance_ns_ks2_5, THR_GOOD)
# df_nb_ks2_5 = quality.qualify_sorted_units(scores_nb_ks2_5, chance_nb_ks2_5, THR_GOOD)
# df_ne_ks2_5 = quality.qualify_sorted_units(scores_ne_ks2_5, chance_ne_ks2_5, THR_GOOD)
df_ds_ks25 = cqadb(THR_GOOD, K25_d1, K25_d2, K25_d3, **k25_sc, **k25_ch)
# # KS2
df_ns_ks2 = quality.qualify_sorted_units(scores_ns_ks2, chance_ns_ks2, THR_GOOD)
# df_nb_ks2 = quality.qualify_sorted_units(scores_nb_ks2, chance_nb_ks2, THR_GOOD)
# df_ne_ks2 = quality.qualify_sorted_units(scores_ne_ks2, chance_ne_ks2, THR_GOOD)
df_ds_ks2 = cqadb(THR_GOOD, K2_d1, K2_d2, K2_d3, **k2_sc, **k2_ch)
# # KS
df_ns_ks = quality.qualify_sorted_units(scores_ns_ks, chance_ns_ks, THR_GOOD)
# df_nb_ks = quality.qualify_sorted_units(scores_nb_ks, chance_nb_ks, THR_GOOD)
# df_ne_ks = quality.qualify_sorted_units(scores_ne_ks, chance_ne_ks, THR_GOOD)
df_ds_ks = cqadb(THR_GOOD, K_d1, K_d2, K_d3, **K_sc, **K_ch)
# # HS
df_ns_hs = quality.qualify_sorted_units(scores_ns_hs, chance_ns_hs, THR_GOOD)
# df_nb_hs = quality.qualify_sorted_units(scores_nb_hs, chance_nb_hs, THR_GOOD)
# df_ne_hs = quality.qualify_sorted_units(scores_ne_hs, chance_ne_hs, THR_GOOD)
df_ds_h = cqadb(THR_GOOD, H_d1, H_d2, H_d3, **H_sc, **H_ch)

In [ ]:
# set experiments
# KS4
df_ns_ks4["sorter"] = "KS4"
df_ns_ks4["experiment"] = "NS"
# df_nb_ks4["sorter"] = "KS4"
# df_nb_ks4["experiment"] = "S"
# df_ne_ks4["sorter"] = "KS4"
# df_ne_ks4["experiment"] = "E"
df_ds_ks4["sorter"] = "KS4"
df_ds_ks4["experiment"] = "DS"

# # KS3
df_ns_ks3["sorter"] = "KS3"
df_ns_ks3["experiment"] = "NS"
# df_nb_ks3["sorter"] = "KS3"
# df_nb_ks3["experiment"] = "S"
# df_ne_ks3["sorter"] = "KS3"
# df_ne_ks3["experiment"] = "E"
df_ds_ks3["sorter"] = "KS3"
df_ds_ks3["experiment"] = "DS"

# # KS2.5
df_ns_ks2_5["sorter"] = "KS2.5"
df_ns_ks2_5["experiment"] = "NS"
# df_nb_ks2_5["sorter"] = "KS2.5"
# df_nb_ks2_5["experiment"] = "S"
# df_ne_ks2_5["sorter"] = "KS2.5"
# df_ne_ks2_5["experiment"] = "E"
df_ds_ks25["sorter"] = "KS2.5"
df_ds_ks25["experiment"] = "DS"

# # KS2
df_ns_ks2["sorter"] = "KS2"
df_ns_ks2["experiment"] = "NS"
# df_nb_ks2["sorter"] = "KS2"
# df_nb_ks2["experiment"] = "S"
# df_ne_ks2["sorter"] = "KS2"
# df_ne_ks2["experiment"] = "E"
df_ds_ks2["sorter"] = "KS2"
df_ds_ks2["experiment"] = "DS"

# # KS
df_ns_ks["sorter"] = "KS"
df_ns_ks["experiment"] = "NS"
# df_nb_ks["sorter"] = "KS"
# df_nb_ks["experiment"] = "S"
# df_ne_ks["sorter"] = "KS"
# df_ne_ks["experiment"] = "E"
df_ds_ks["sorter"] = "KS"
df_ds_ks["experiment"] = "DS"

# # HS
df_ns_hs["sorter"] = "HS"
df_ns_hs["experiment"] = "NS"
# df_nb_hs["sorter"] = "HS"
# df_nb_hs["experiment"] = "S"
# df_ne_hs["sorter"] = "HS"
# df_ne_hs["experiment"] = "E"
df_ds_h["sorter"] = "HS"
df_ds_h["experiment"] = "DS"

# concatenate
df = pd.concat(
    [
        df_ns_ks4,
        # df_nb_ks4,
        # df_ne_ks4,
        df_ns_ks3,
        # df_nb_ks3,
        # df_ne_ks3,
        df_ns_ks2_5,
        # df_nb_ks2_5,
        # df_ne_ks2_5,
        df_ns_ks2,
        # df_nb_ks2,
        # df_ne_ks2,
        df_ns_ks,
        # df_nb_ks,
        # df_ne_ks,
        df_ns_hs,
        # df_nb_hs,
        # df_ne_hs,
    ]
)
df = df.sort_values("quality")

# temp save
# df.to_csv("sorting_quality.csv", index=False)

In [28]:
# read pre-computed data
# df = pd.read_csv(quality_path)

### Plot by experiment x sorter

In [12]:
# plot - kilosort 4
ax = quality.plot_ratio_by_exp(df[df["sorter"] == "KS4"], cl, legend_cfg)

# plot - kilosort 3
ax = quality.plot_ratio_by_exp(df[df["sorter"] == "KS3"], cl, legend_cfg)

# plot - kilosort 2.5
ax = quality.plot_ratio_by_exp(df[df["sorter"] == "KS2.5"], cl, legend_cfg)

# plot - kilosort 2
ax = quality.plot_ratio_by_exp(df[df["sorter"] == "KS2"], cl, legend_cfg)

# plot - kilosort
ax = quality.plot_ratio_by_exp(df[df["sorter"] == "KS"], cl, legend_cfg)

# plot - herdingspikes
ax = quality.plot_ratio_by_exp(df[df["sorter"] == "HS"], cl, legend_cfg)

NameError: name 'df' is not defined

### References

(1) https://neuronaldynamics.epfl.ch/online/Ch7.S2.html